In [1]:
import numpy as np
import scipy
from qutip import *
from numba import jit, njit

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap

from mpl_toolkits.mplot3d import Axes3D
from plotly.offline import plot
import plotly.graph_objs as go

In [68]:
sx = sigmax()/2
sy = sigmay()/2
sz = sigmaz()/2
s = sx - 1j*sy 

sig1 = tensor(s, qeye(2), qeye(2))
sig2 = tensor( qeye(2), s, qeye(2))
sig3 = tensor( qeye(2), qeye(2), s)
sigs = [sig1, sig2, sig3]

flip_t = (sig1.dag()+sig1)*(sig2.dag()+sig2)*(sig3.dag()+sig3)

H0 = sig1.dag()*sig1 + sig2.dag()*sig2 + sig3.dag()*sig3
evals, evecs = H0.eigenstates(sort='low')

vac = evecs[0]
ups = evecs[len(evals)-1]

#M = np.array( [ [0,np.exp(1j*np.pi/3),np.exp(-1j*np.pi/3)], [np.exp(-1j*np.pi/3),0,np.exp(1j*np.pi/3)], [np.exp(1j*np.pi/3),np.exp(1j*np.pi/3),0] ] )
M = np.array( [ [1,1,1], [1,1,1], [1,1,1] ] )*2.0
#evalsM, evecsM = np.linalg.eig(M)
evalsM = np.array([1,0,0])
vec1 = np.array([1,1,1]) /3**.5
vec2 = np.array([ -2**.5, 1/2**.5, 1/2**.5]) /3**.5
vec3 = np.array([ 1/2**.5, 1/2**.5,  -2**.5 ]) /3**.5
vec3 = vec3 - vec2 * np.dot(vec2, vec3)
vec3 = vec3/np.sqrt(np.dot(vec3,vec3))

evecsM = M.copy()*0.0
evecsM[:,0] = vec1
evecsM[:,1] = vec2
evecsM[:,2] = vec3
#evecsM = np.array( [[1,-2**.5,1/2**.5], [1,1/2**.5,1/2**.5], [1,1/2**.5,-2**.5]]  )/3**.5

print(evecsM, np.abs(evalsM), np.matmul(evecsM.T, evecsM ) )

fs = []
for n in range(len(evalsM)):
    f = 0*sig1
    for m in range(len(evalsM)):
        f += sigs[m]*evecsM[m][n]
    fs.append(f)
        

[[ 5.77350269e-01 -8.16496581e-01 -1.92296269e-16]
 [ 5.77350269e-01  4.08248290e-01  7.07106781e-01]
 [ 5.77350269e-01  4.08248290e-01 -7.07106781e-01]] [1 0 0] [[ 1.00000000e+00 -4.87077901e-17 -1.37406100e-16]
 [-4.87077901e-17  1.00000000e+00  1.29464217e-16]
 [-1.37406100e-16  1.29464217e-16  1.00000000e+00]]


In [89]:
P1 = fs[0]*ups
P1 = P1/np.sqrt(np.abs( P1.overlap(P1)) )

P12 = fs[1]*P1
P12 = P12/np.sqrt( np.abs(P12.overlap(P12)) )

P123 = fs[2]*P12
#P123 = P123/np.sqrt( np.abs(P123.overlap(P123)) )

P13 = fs[2]*P1
P13 = P13/np.sqrt(P13.overlap(P13)) 

P111 = fs[0]*P13
#P111 = P111/np.sqrt(P111.overlap(P111))

S1 = entropy_vn( ( P1 * P1.dag() ).ptrace(0) )
S12 = entropy_vn( ( P12 * P12.dag() ).ptrace(0) )
S123 = entropy_vn( ( P123 * P123.dag() ).ptrace(0) )
S13 = entropy_vn( ( P13 * P13.dag() ).ptrace(0) )
S111 = entropy_vn( ( P111 * P111.dag() ).ptrace(0) )

N1 = expect(H0, P1)
N12 = expect(H0, P12)
N123 = expect(H0, P123)
N13 = expect(H0, P13)
N111 = expect(H0, P111)



print(S1, S12, S13)
print(N1, N12, N13)
print(P1.overlap( flip_t* fs[0]*P1))


0.6365141682948128 0.6365141682948129 2.220446049250313e-16
2.0000000000000004 1.0 0.9999999999999998
(1.1547005383792521+0j)
